## Adjust bathymetry to Coastline 

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
from pyresample import bilinear, geometry, kd_tree
from pyresample import utils
import pyproj


In [ ]:
from pyPoseidon.dem import *
from pyPoseidon.utils.fix import *

In [ ]:
class Basemap(Basemap):
    """ Modify Basemap to use Natural Earth data instead of GSHHG data """
    def drawcoastlines(self):
        shapefile = '/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_%sm_coastline' % \
                    {'l':110, 'i':50, 'h':10}[self.resolution]
        self.readshapefile(shapefile, 'coastline', linewidth=1.)
#    def drawcountries(self):
#        shapefile = '/Users/brey/Downloads/data/naturalearth/countries/ne_%sm_admin_0_countries' % \
#                    {'l':110, 'm':50, 'h':10}[self.resolution]
#        self.readshapefile(shapefile, 'countries', linewidth=0.5)




In [ ]:
dic = {'minlon' : 0., # lat/lon window
       'maxlon' : 13.,
       'minlat' : 54.5,
       'maxlat' : 66.5,
       'dem':'emodnet',
       'dpath':'/Users/brey/DATA/emodnet_bathymetry.nc'}

In [ ]:
dic = {'minlon' : 0., # lat/lon window
       'maxlon' : 13.,
       'minlat' : 54.5,
       'maxlat' : 66.5,
       'dem':'gebco',
       'dpath':'/Users/brey/DATA/GEBCO_2014_2D.nc'}

## Compute bathymetry

In [ ]:
if dic['dem']=='emodnet':
    b = emodnet(**dic)
elif dic['dem']=='gebco':
    b = gebco(**dic)

In [ ]:
b.Dataset

In [ ]:
b.Dataset.val.plot()

##  Test pyresample for given grid

### Make grid

In [ ]:
## lat,lon grid
lon=np.arange(dic['minlon'],dic['maxlon'],.1)
lat=np.arange(dic['minlat'],dic['maxlat'],.1)
x, y = np.meshgrid(lon,lat)

### Test 1 - ImageContainerNearest

In [ ]:
lons = b.Dataset.dlons.values
lats = b.Dataset.dlats.values
bath = b.Dataset.val.values

In [ ]:
orig = pyresample.geometry.SwathDefinition(lons=lons,lats=lats) # original points
targ = pyresample.geometry.SwathDefinition(lons=x,lats=y) # target grid

In [ ]:
grid_con = pyresample.image.ImageContainerNearest(bath, orig, radius_of_influence=50000,fill_value=np.nan,nprocs=4)

In [ ]:
area_con = grid_con.resample(targ)

In [ ]:
result1 = area_con.image_data

### Test 2 - kd_tree.resample_nearest

In [ ]:
lons = b.Dataset.dlons.values
lats = b.Dataset.dlats.values
bath = b.Dataset.val.values

In [ ]:
orig = pyresample.geometry.SwathDefinition(lons=lons,lats=lats) # original points
targ = pyresample.geometry.SwathDefinition(lons=x,lats=y) # target grid

In [ ]:
result2 = pyresample.kd_tree.resample_nearest(orig,bath,targ,radius_of_influence=50000,fill_value=np.nan,nprocs=4)

In [ ]:
## Compare 
np.array_equal(result1,result2)

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result1)
m.scatter(x,y,c=result2)


m.colorbar()

### Test 3 - bilinear.resample_bilinear

In [ ]:
#Set lat/lon window for interpolation
prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
[[a0,a1],[a2,a3]] = prj([dic['minlon'], dic['minlat']], [dic['maxlon'], dic['maxlat']])

a0,a1,a2,a3

In [ ]:
target_def = pyresample.geometry.AreaDefinition('', '', '',
                                {
                                 'lat_0': '0.00', 'lat_ts': '0.00',
                                 'lon_0': '0.00', 'proj': 'eqc'},
                                x.shape[1], x.shape[0],
                                [a0, a1,
                                 a2, a3])

print(target_def)

In [ ]:
source_def = pyresample.geometry.SwathDefinition(lons=lons,lats=lats) # original bathymetry points

In [ ]:
result3 = bilinear.resample_bilinear(bath, source_def, target_def,
                                      radius=50e3, neighbours=32,
                                      nprocs=1, fill_value=np.nan,
                                      reduce_data=True, segments=None,
                                      epsilon=0)


In [ ]:
## Compare 
np.array_equal(result1,result3)

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result1)
m.scatter(x,y,c=np.flipud(result3))


m.colorbar()

In [ ]:
dif = result1 - np.flipud(result3)

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result1)
m.scatter(x,y,c=dif)


m.colorbar()

## use the shoreline

### Make grid

In [ ]:
## lat,lon grid
lon=np.arange(dic['minlon'],dic['maxlon'],.1)
lat=np.arange(dic['minlat'],dic['maxlat'],.1)
x, y = np.meshgrid(lon,lat)

In [ ]:
dic.update({'grid_x':x, 'grid_y':y})

In [ ]:
points=zip(x.flatten(),y.flatten())

### Get Bathymetry

In [ ]:
if dic['dem']=='emodnet':
    b = emodnet(**dic)
elif dic['dem']=='gebco':
    b = gebco(**dic)

In [ ]:
shapefile = '/Users/brey/DATA/COASTLINES/naturalearth/coastline/ne_%sm_coastline.shp' % \
                    {'l':110, 'i':50, 'h':10}['i']

In [ ]:
wmask, cg = fix(b,shapefile)

In [ ]:
xp = b.Dataset.ilons.values
yp = b.Dataset.ilats.values
          
##resample bathymetry
xw=np.ma.masked_array(xp,wmask) #wet points 
yw=np.ma.masked_array(yp,wmask) 




In [ ]:
%matplotlib notebook

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result1)
m.scatter(xw,yw,c=b.Dataset.ival)


m.colorbar()

## Adjust bathymetry

### Test 1 - ImageContainerNearest

In [ ]:
xp = b.Dataset.ilons.values
yp = b.Dataset.ilats.values
          
##resample bathymetry
xw=np.ma.masked_array(xp,wmask) #wet points 
yw=np.ma.masked_array(yp,wmask) 




In [ ]:
b.Dataset.val.data[np.isnan(b.Dataset.val)]=9999. # fill the nan, if present, we values in order to compute values there if needed.

In [ ]:
wet = np.ma.masked_array(b.Dataset.val,b.Dataset.val>0)

In [ ]:



mx = np.ma.masked_array(b.Dataset.dlons,b.Dataset.val>0) 
my = np.ma.masked_array(b.Dataset.dlats,b.Dataset.val>0)





In [ ]:
orig = pyresample.geometry.SwathDefinition(lons=mx,lats=my) # original bathymetry points

In [ ]:
targ = pyresample.geometry.SwathDefinition(lons=xw,lats=yw) # grid bathymetry points

Note that it works also with pyresample.geometry.GridDefinition but it gives a warning due to Nan masking. The results are the same

In [ ]:
grid_con = pyresample.image.ImageContainerNearest(wet, orig, radius_of_influence=50000,fill_value=np.nan)

area_con = grid_con.resample(targ)

result1 = area_con.image_data

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result1)
m.scatter(xw,yw,c=result1)


m.colorbar()

In [ ]:
dif1 = b.Dataset.ival - result1

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
m.contourf(xw,yw,dif1)
m.colorbar()

### Test 2 - kd_tree.resample_nearest

In [ ]:
orig = pyresample.geometry.SwathDefinition(lons=mx,lats=my) # original bathymetry points

In [ ]:
targ = pyresample.geometry.SwathDefinition(lons=xw,lats=yw) # original bathymetry points

In [ ]:
result2 = pyresample.kd_tree.resample_nearest(orig, wet, targ, radius_of_influence=50000,epsilon=0.5, fill_value=np.nan)

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result2)
m.scatter(xw,yw,c=result2)
m.colorbar()

In [ ]:
dif2 = b.Dataset.ival - result2

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
m.contourf(xw,yw,dif2)
m.colorbar()

### Test 3 - kd_tree.resample_gauss

In [ ]:
orig = pyresample.geometry.SwathDefinition(lons=mx,lats=my) # original bathymetry points

In [ ]:
targ = pyresample.geometry.SwathDefinition(lons=xw,lats=yw) # original bathymetry points

In [ ]:
result3 = pyresample.kd_tree.resample_gauss(orig, wet, targ, radius_of_influence=50000,sigmas=25000,fill_value=np.nan)

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result3)
m.scatter(xw,yw,c=result3)
m.colorbar()

In [ ]:
dif3 = b.Dataset.ival - result3

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
m.contourf(xw,yw,dif3)
m.colorbar()

#### Test 4 - bilinear.resample_bilinear

In [ ]:
#Set lat/lon window for interpolation
prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')
[[a0,a1],[a2,a3]] = prj([dic['minlon'], dic['minlat']], [dic['maxlon'], dic['maxlat']])

a0,a1,a2,a3

In [ ]:
area_id = ''
description = ''
proj_id = ''
projection = '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
x_size = xw.shape[1]
y_size = xw.shape[0]
area_extent = (a0, a1, a2, a3)
target_def = utils.get_area_def(area_id, description, proj_id, projection,
                                       x_size, y_size, area_extent)
print(target_def)



.. or ...

In [ ]:
target_def = pyresample.geometry.AreaDefinition('', '', '',
                                {
                                 'lat_0': '0.00', 'lat_ts': '0.00',
                                 'lon_0': '0.00', 'proj': 'eqc'},
                                xw.shape[1], xw.shape[0],
                                [a0, a1,
                                 a2, a3])

print(target_def)

In [ ]:
source_def = pyresample.geometry.SwathDefinition(lons=mx,lats=my) # original bathymetry points

In [ ]:
result4 = bilinear.resample_bilinear(wet, source_def, target_def,
                                      radius=50e3, neighbours=32,
                                      nprocs=1, fill_value=np.nan,
                                      reduce_data=True, segments=None,
                                      epsilon=0)


In [ ]:
result4 = np.flipud(result4) # WHY?

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
#m.contourf(xw,yw,result4)
m.scatter(xw,yw,c=result4)
m.colorbar()

In [ ]:
dif4 = b.Dataset.ival - result4

In [ ]:
plt.figure()
m = Basemap(llcrnrlon=dic['minlon'], llcrnrlat=dic['minlat'], urcrnrlon=dic['maxlon'], urcrnrlat=dic['maxlat'],
            resolution='i')  # resolution = (l)ow | (i)intermediate | (h)igh
m.drawcoastlines()
m.contourf(xw,yw,dif4)
m.colorbar()

NOTE that bilinear doesn't work here due to Nan values